1. __Bag of words__  
2. __te_smming__  
3. __unigram, bi-gram,n-gram__  
4. __Tf-Idf (term frequency, inverse document frequency)__  
5. __Word2vec__  
6. __Avg - Word2vec and weighted 2ve__

__Features Information :__  
  
  1. Id
  2. ProductId - unique identifier for the product
      - For each product their is unique_id.
      - In the URL, I discovered a 'dp' (data page) through diligent research, yielding valuable insights specifically for Amazon. However, details for other websites remain undisclosed.

  3.  UserId - unqiue identifier for the user
      - A user, having purchased or gained knowledge about a product, eligible to submit a review.
  4. ProfileName
      - When submitting a review, your name will be displayed alongside your feedback
  5. HelpfulnessNumerator - number of users who found the review helpful
  6. HelpfulnessDenominator - number of users who indicated whether they found the review helpful or not
  7. Score - rating between 1 and 5
  8. Time - timestamp for the review
  9. Summary - brief summary of the review
  10. Text - text of the review

In [1]:
import warnings
warnings.filterwarnings("ignore")
import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer

import re
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pickle

from tqdm import tqdm
import os

## Connection for SQLLite

- We are utilizing SQLite for our database operations. 
  Initially, we establish a connection to SQLite by using the sqlite3 module and specifying the database name.
- Following successful interaction between the interpreter and the downloaded file, a SQLite3 connection is established
- Now, we have the capability to interact with the database and retrieve data using SQL commands.

In [2]:
conn = sqlite3.connect('database.sqlite')
type(conn)

sqlite3.Connection

we can pandas tto write sqlquerry inorder to fetch the data

In [3]:
df = pd.read_sql_query("""select * from Reviews where Score != 3""", conn)

- Products receiving a score of 1 or 2 are categorized as negative reviews.
- Products earning a score of 4 or 5 are categorized as positive reviews.
- A product with a score of 3 is neutral initially; however, its sentiment (positive or negative) will be determined through further text analysis of the user's review.

In [4]:
print(df.shape)
df.head()

(525814, 10)


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


- Most servers commonly use Unix time, which is a representation of time in seconds since the Unix epoch. They often prefer Unix time and may not explicitly maintain separate date and time values
- to chech the unixttime [UNIXTIME](https://www.unixtimestamp.com/)

Created function to change score Pos(+ve), Neg(-ve)

In [5]:
def sep(score):
    if score < 3:
        return 'Negative'
    else:
        return 'Positiive'

In [6]:
df['Score'] = df['Score'].map(sep)

print(df.shape)
df.head()

(525814, 10)


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,Positiive,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,Negative,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,Positiive,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,Negative,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,Positiive,1350777600,Great taffy,Great taffy at a great price. There was a wid...


## Data Cleaning

Find Duplicates

In [7]:
pd.read_sql_query("""SELECT UserId, Time, COUNT(*)
FROM REVIEWS
GROUP BY UserId, Time
HAVING COUNT(*)>1
ORDER BY COUNT(*) DESC""",conn)

,UserId,Time,COUNT(*)
0,A3TVZM3ZIXG8YW,1291420800,199
1,A29JUMRL1US6YP,1278201600,125
2,AJD41FBJD9010,1233360000,73
3,ABDCYK04CL6O4,1323993600,68
4,A26NFIQ7KWI8Y7,1329696000,65
...,...,...,...
76852,AZZA4Q0JACD5U,1340582400,2
76853,AZZH3GGYQSBUC,1229299200,2
76854,AZZTH6DJ0KSIP,1304208000,2
76855,AZZU4D6TZ2L6J,1247875200,2


In [8]:
df[(df['UserId'] == 'A3TVZM3ZIXG8YW') & (df['Time'] == 1291420800)]

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
2702,2942,B0002TJAZK,A3TVZM3ZIXG8YW,christopher hayes,7,11,Negative,1291420800,"Filler food is empty, leaves your cat always n...","This review will make me sound really stupid, ..."
2707,2948,B0002TJAZK,A3TVZM3ZIXG8YW,christopher hayes,0,2,Negative,1291420800,"Filler food is empty, leaves your cat always n...","This review will make me sound really stupid, ..."
29149,31783,B00106TG9Y,A3TVZM3ZIXG8YW,christopher hayes,2,8,Negative,1291420800,"Filler food is empty, leaves your cat always n...","This review will make me sound really stupid, ..."
48303,52497,B003ANFMY8,A3TVZM3ZIXG8YW,christopher hayes,19,21,Negative,1291420800,"Filler food is empty, leaves your cat always n...","This review will make me sound really stupid, ..."
48307,52502,B003ANFMY8,A3TVZM3ZIXG8YW,christopher hayes,18,24,Negative,1291420800,"Filler food is empty, leaves your cat always n...","This review will make me sound really stupid, ..."
...,...,...,...,...,...,...,...,...,...,...
462311,499917,B009B87SAC,A3TVZM3ZIXG8YW,christopher hayes,6,14,Negative,1291420800,"Filler food is empty, leaves your cat always n...","This review will make me sound really stupid, ..."
462312,499918,B009B87SAC,A3TVZM3ZIXG8YW,christopher hayes,6,15,Negative,1291420800,"Filler food is empty, leaves your cat always n...","This review will make me sound really stupid, ..."
475425,514141,B003M5VM8O,A3TVZM3ZIXG8YW,christopher hayes,5,9,Negative,1291420800,"Filler food is empty, leaves your cat always n...","This review will make me sound really stupid, ..."
475946,514691,B003MWGSKY,A3TVZM3ZIXG8YW,christopher hayes,3,8,Negative,1291420800,"Filler food is empty, leaves your cat always n...","This review will make me sound really stupid, ..."


- looks like "christopher hayes" as done 199 review same time for the products
- if you see the first 2 product it as same productid and review for all are quite same

In [9]:
# pd.read_sql_query("""
# select * from Reviews 
# where Score != 3 and UserId = 'AR5J8UI46CURR'
# order by ProductId""", conn)

In [12]:
df_dup = df.drop_duplicates(subset = ["UserId","Time","Text"])

# remaining percentage loss
percentage_remaining = (df_dup['Id'].shape[0] / df['Id'].shape[0])*100

print(f' Shape of original data: {df.shape[0]}\n Shape of final data after "Duplicates Removed": {df_dup.shape[0]}\n')
print(f' The final df represents approximately {percentage_remaining:.2f}% of the original dataset after removing duplicates.')


 Shape of original data: 525814
 Shape of final data after "Duplicates Removed": 364133

 The final df represents approximately 69.25% of the original dataset after removing duplicates.


In [13]:
df_dup[(df_dup['HelpfulnessNumerator'] > df_dup['HelpfulnessDenominator'])]

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
41159,44737,B001EQ55RW,A2V0I904FH7ABY,Ram,3,2,Positiive,1212883200,Pure cocoa taste with crunchy almonds inside,It was almost a 'love at first bite' - the per...
59301,64422,B000MIDROQ,A161DK06JJMCYF,"J. E. Stephens ""Jeanne""",3,1,Positiive,1224892800,Bought This for My Son at College,My son loves spaghetti so I didn't hesitate or...


Observe that in the dataset, there are instances where HelpfulnessNumerator exceeds HelpfulnessDenominator.  
Recognize that this inconsistency indicates a potential error or issue in the data collection process.  
Consider addressing this issue as a form of data error or duplication.  
Note that removing such instances may not significantly affect the overall dataset, as these cases likely represent errors in the voting information.


In [14]:
df_final = df_dup[(df_dup['HelpfulnessNumerator'] <= df_dup['HelpfulnessDenominator'])]
print("Remaining data left: ",df_final.shape[0])

Remaining data left:  364131


In [15]:
df_final['Score'].value_counts()

Positiive    307026
Negative      57105
Name: Score, dtype: int64